In [1]:
# Import Dependancies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from config import weather_api_key
import requests
#Create a URL variable for the API request
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of random latitudes and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list for query
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding cities near coordinates
cities = []
# Identify the nearest city to the coordinates using citipy module
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    #Evaluate if the city is unique and add to the city list
    if city not in cities:
        cities.append(city)
# Print the count of cities collected in the city list
len(cities)


744

In [5]:
# Import the time library and the datetime module from the datetime library for timer
import time
from datetime import datetime

In [16]:
# Create a list to hold the weather data from the API query
city_data = []

# Create counters for request loops
record_count = 1
set_count = 1

# Print initiating text for the request query loop
print("Beginning data collection")
print("-------------------------")

# Loop through cities in the cities list
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for API request criteria
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create request URL for each city
    city_url = url + "&q=" + city.replace(" ", "+")
      
    #Log in using the URL and collect and record the set number and city.
    print(f"Processing {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count variable
    record_count += 1
    
# Run API request with error management for each city in the cities list
    try:
        # Parse the JSON and retrieve the needed data
        city_weather = requests.get(city_url).json()
        city_weather
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into the city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
        
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
        
#Indicate the data loading is completed.
print("-----------------------")
print("Data Retrieval Complete")
print("-----------------------")


Beginning data collection
-------------------------
Processing 1 of Set 1 | bluff
Processing 2 of Set 1 | torbay
Processing 3 of Set 1 | rikitea
Processing 4 of Set 1 | tasiilaq
Processing 5 of Set 1 | neiafu
Processing 6 of Set 1 | ushuaia
Processing 7 of Set 1 | avarua
Processing 8 of Set 1 | butaritari
Processing 9 of Set 1 | vaini
Processing 10 of Set 1 | jamestown
Processing 11 of Set 1 | rapar
Processing 12 of Set 1 | puerto ayora
Processing 13 of Set 1 | pangnirtung
Processing 14 of Set 1 | malumfashi
Processing 15 of Set 1 | nishihara
Processing 16 of Set 1 | east london
Processing 17 of Set 1 | barrow
Processing 18 of Set 1 | albany
Processing 19 of Set 1 | kavaratti
Processing 20 of Set 1 | kapaa
Processing 21 of Set 1 | bialystok
Processing 22 of Set 1 | arraial do cabo
Processing 23 of Set 1 | valleyview
Processing 24 of Set 1 | saskylakh
Processing 25 of Set 1 | namibe
Processing 26 of Set 1 | suntar
Processing 27 of Set 1 | xining
Processing 28 of Set 1 | cherskiy
Process

In [17]:
len(city_data)


679

In [18]:
# Convert city_data list to a DataFrame
city_data_df = pd.DataFrame(city_data)

new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Bluff,NZ,-46.6000,168.3333,49.87,91,100,10.60,light rain
1,Torbay,CA,47.6666,-52.7314,52.75,99,90,21.85,light rain
2,Rikitea,PF,-23.1203,-134.9692,76.87,72,8,10.40,clear sky
3,Tasiilaq,GL,65.6145,-37.6368,26.76,56,100,8.75,overcast clouds
4,Neiafu,TO,-18.6500,-173.9833,84.88,70,75,6.91,broken clouds
...,...,...,...,...,...,...,...,...,...
674,Pushchino,RU,54.8339,37.6264,35.35,87,52,16.13,broken clouds
675,Baghmara,IN,23.7950,86.2085,66.09,41,0,3.40,clear sky
676,Launceston,AU,-41.4500,147.1667,64.15,48,75,8.99,broken clouds
677,Necochea,AR,-38.5473,-58.7368,61.03,78,2,6.29,clear sky


In [20]:
#Create the output file (CSV)
output_data_file = "../Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")